# Imports and dataset loading

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

import pandas as pd

strategyArray = ["Random Frest", "BRET"]
depthArray = np.arange(2, 16)
gainArray = []
lossArray = []

gainRF = []
lossRF = []
gainBRET = []
lossBRET = []

originalDataSet = pd.read_csv('dataset', sep=',',header = None)
scaler = MinMaxScaler()

def loadDataset():    
    scaler.fit(originalDataSet.sample(frac=1).values.reshape(-1, 1))
    dataSet = scaler.transform(originalDataSet.values) 
    
    features, result = np.array(dataSet[:, :4]), np.array(dataSet[:, 4:])
    return features, result

def invertNorm(value):
    auxArray = np.array([value, 0, 0, 0, 0, 0]).reshape(-1, 1)
    return scaler.inverse_transform(auxArray)[0][0]
    
def getError(value1, value2):
    return abs(invertNorm(value1) - invertNorm(value2))

# Cross validation function

In [9]:
def crossValidate(regressor, features, result, folds = 5):
    foldSize   = math.ceil(features.shape[0] / folds)
    gainErrors = []
    lossErrors = []
    
    for i in range(folds): 
        sliceBegin = i * foldSize
        sliceEnd   = (i + 1) * foldSize
        
        X_train = np.delete(features, np.s_[sliceBegin: sliceEnd], 0)
        y_train = np.delete(result, np.s_[sliceBegin: sliceEnd], 0)
        
        regressor.fit(X_train, y_train)    
            
        X_test = features[sliceBegin: sliceEnd]
        y_test = result[sliceBegin: sliceEnd]
        
        gainError = 0
        lossError = 0
        
        prediction = regressor.predict(X_test)
        
        for predicted, expected in zip(prediction, y_test):
            #print("Gain: ", invertNorm(predicted[0]), round(invertNorm(expected[0]), 14))
            #print("Loss: ", invertNorm(predicted[1]), round(invertNorm(expected[1]), 14))
            gainError += getError(predicted[0], expected[0])
            lossError += getError(predicted[1], expected[1])
            
        gainErrors.append(gainError / foldSize)
        lossErrors.append(lossError / foldSize)

    return np.array(gainErrors), np.array(lossErrors)

In [10]:
def trainModel(strategy):    
    features, result = loadDataset()
    gainErrors, lossErrors = crossValidate(strategy, features, result)

    print(gainErrors, "=> %0.2f (+/- %0.2f)" % (np.mean(gainErrors), gainErrors.std() * 2))
    print(lossErrors, "=> %0.2f (+/- %0.2f)" % (np.mean(lossErrors), lossErrors.std() * 2))
    
    if isinstance(strategy, RandomForestRegressor):
        gainRF.append(np.mean(gainErrors))
        lossRF.append(np.mean(lossErrors))
    else:
        gainBRET.append(np.mean(gainErrors))
        lossBRET.append(np.mean(lossErrors))
        

# Plotting functions

In [11]:
def plotGainError():
    fig7, axis = plt.subplots(figsize = (10, 5))

    axis.plot(depthArray, gainRF, 'sb-', depthArray, gainBRET, 'or--')
    axis.set_title("Gain error")
    axis.set_ylabel("Absolute error")
    axis.set_ylabel("Max depth")
    axis.legend(strategyArray)

    plt.show()

In [12]:
def plotLossError():
    fig7, axis = plt.subplots(figsize = (10, 5))

    axis.plot(depthArray, lossRF, 'sb-', depthArray, lossBRET, 'or--')
    axis.set_title("Loss error")
    axis.set_ylabel("Absolute error")
    axis.set_ylabel("Max depth")
    axis.legend(strategyArray)

    plt.show()

# Putting all together

In [13]:
for depth in depthArray:
    randomForest = RandomForestRegressor(max_depth = depth, random_state = 0, n_estimators = 100)
    trainModel(randomForest)

print("Done!")

[3.05147612 2.83995014 2.77142105 3.01725143 2.8654347 ] => 2.91 (+/- 0.21)
[2.06537041 1.6955493  1.85067958 1.60601904 1.20084246] => 1.68 (+/- 0.57)
[2.79104018 2.71649607 2.72778047 2.97641909 2.75170767] => 2.79 (+/- 0.19)
[2.03894986 1.85040552 1.8520129  2.00712057 1.24182475] => 1.80 (+/- 0.58)
[2.74504597 2.66109393 2.65683641 2.72590319 2.83384356] => 2.72 (+/- 0.13)
[1.99764764 1.93266278 1.84299334 1.95440677 1.19545335] => 1.78 (+/- 0.60)
[2.52908705 2.60426109 2.57169172 2.61446418 2.69697152] => 2.60 (+/- 0.11)
[1.91634158 1.81347573 1.82714762 1.93883567 1.25121989] => 1.75 (+/- 0.51)
[2.43446871 2.46280751 2.47538524 2.54855339 2.5797798 ] => 2.50 (+/- 0.11)
[1.90628606 1.79893034 1.78399062 1.98103164 1.20866749] => 1.74 (+/- 0.55)
[2.29879112 2.3548694  2.38642412 2.46689141 2.59639857] => 2.42 (+/- 0.21)
[1.94740003 1.75961494 1.73198172 2.00622993 1.24275722] => 1.74 (+/- 0.54)
[2.16927505 2.25317474 2.29048095 2.42384935 2.52402302] => 2.33 (+/- 0.25)
[2.00369648 

In [ ]:
for depth in depthArray:
    extra_tree = ExtraTreeRegressor(criterion = "mae", max_depth = depth, random_state=0)
    bret = MultiOutputRegressor(BaggingRegressor(base_estimator=extra_tree, n_estimators=100, random_state=0))
    trainModel(bret)

print("Done!")

[2.8972063  2.92091691 2.57318052 3.04395415 2.68545845] => 2.82 (+/- 0.34)
[2.06697708 1.45989971 1.8217192  1.4104298  1.15525788] => 1.58 (+/- 0.64)
[2.72213467 2.80845272 2.53538682 2.96489971 2.66146132] => 2.74 (+/- 0.29)
[2.05315186 1.44961318 1.8132808  1.40491404 1.1834957 ] => 1.58 (+/- 0.62)
[2.6377937  2.68895415 2.45008596 2.8624212  2.60939828] => 2.65 (+/- 0.27)
[2.01783668 1.44297994 1.7980659  1.39265043 1.19942693] => 1.57 (+/- 0.59)
[2.51958453 2.54472779 2.43222063 2.76621777 2.51553009] => 2.56 (+/- 0.22)
[1.98282235 1.43681948 1.77054441 1.36989971 1.19968481] => 1.55 (+/- 0.57)
[2.4213467  2.44054441 2.36775072 2.68723496 2.49289398] => 2.48 (+/- 0.22)
[1.9615616  1.43886819 1.77160458 1.34133238 1.22641834] => 1.55 (+/- 0.55)
[2.3445702  2.3562894  2.28560172 2.57346705 2.47690544] => 2.41 (+/- 0.21)
[1.95929799 1.46594556 1.74570201 1.34055874 1.26561605] => 1.56 (+/- 0.52)
[2.33873926 2.27246418 2.25561605 2.52448424 2.48908309] => 2.38 (+/- 0.22)
[1.97181948 

# Plottin error boxplots

In [ ]:
plotGainError()
plotLossError()